# Preprocessing Titanic #

libraries needed

In [22]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import re 

import data

In [23]:
df = pd.read_csv('train.csv', index_col='PassengerId')
titanic = pd.read_csv('train.csv')#, index_col='PassengerId')
df_test = pd.read_csv('test.csv' , index_col='PassengerId')
test_titanic = pd.read_csv('test.csv')# , index_col='PassengerId')

drop  useless feature

In [24]:
titanic.drop(['Name','Ticket', 'PassengerId'], axis=1, inplace=True)

test_titanic.drop(['Name','Ticket','PassengerId'], axis=1 , inplace=True)

print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 62.7+ KB
None


feature cabin --> keeping the deck & filling NA with 'Z'

In [25]:
print(titanic.isna().sum())
print(test_titanic.isna().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64
Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64


In [26]:
titanic["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'Z' for i in titanic['Cabin'] ])

test_titanic['Cabin'].fillna('Z', inplace=True)
#test_titanic["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'Z' for i in test_titanic['Cabin'] ])
Cabin = []
for i in test_titanic['Cabin']:
    Cabin.append(i[0])
test_titanic['Cabin'] = Cabin



Filling NA in age by mean by Pclass

In [27]:
means = df.groupby('Pclass').Age.median()
for i in titanic[titanic.Age.isnull()].index:
    titanic.loc[i, 'Age'] = means[titanic.loc[i].Pclass]
    
for i in test_titanic[test_titanic.Age.isnull()].index:
    test_titanic.loc[i, 'Age'] = means[test_titanic.loc[i].Pclass]

Get title from the name feature and aggregate similar value

In [29]:
df_title = df['Name']
# Get Title from Name
data_title = [i.split(",")[1].split(".")[0].strip() for i in df_title]
titanic["Title"] = pd.Series(data_title)
titanic['Title'].replace({'Mme':'Mrs','the Countess':'Lady','Mlle':'Miss','Ms':'Mrs' ,'Jonkheer':'Sir', 'Major':'Col', 'Capt':'Col' , 'Don':'Sir'}, inplace=True)

df_title = df_test['Name']
data_title = [i.split(",")[1].split(".")[0].strip() for i in df_title]
test_titanic['Title'] = data_title
test_titanic['Title'].replace({'Dona':'Lady', 'Ms' : 'Mrs'}, inplace=True)

fill na in the feature fare in the test data by 0 and drop the 2 rows with na in 'embarked' in the train set

In [30]:
test_titanic['Fare'].fillna(0, inplace=True)
titanic.dropna(inplace=True)


print(test_titanic.isna().sum())
print(titanic.isna().sum())

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
Title       0
dtype: int64
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
Title       0
dtype: int64


remove target from train datas

In [32]:
X = titanic.drop('Survived', axis=1)
y = titanic['Survived']

label encoder

training model

In [34]:
randomfor = RandomForestClassifier(n_jobs= -1)

paramsrf={
        'n_estimators': [200],#[50,100,200],
        'bootstrap' : [True],# False],
#        'oob_score' : [True, False],
        'warm_start' : [False], #, False],
        'class_weight' : ['balanced'],#['balanced', 'balanced_subsample', None]
        'max_depth' : [4]        
        }
gridforest = GridSearchCV(estimator=randomfor, cv=10, param_grid=paramsrf, pre_dispatch=4, verbose=1, scoring='accuracy')   
gridforest.fit(X,y)

bestforest = gridforest.best_estimator_
predictforest = bestforest.predict(X)
reportforest = classification_report(y, predictforest)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.4s finished


predict on the test data and formatting the csv for kaggle

In [35]:
predict_ts= bestforest.predict(test_titanic)
predict_ts = pd.DataFrame(predict_ts)
predict_ts.columns = ['Survived']
predict_ts.index = df_test.index
predict_ts.index.name = 'PassengerId'
predict_ts.to_csv('titanic.csv')